In [10]:

import urllib
import xml.etree.ElementTree as ET
import urllib
import re
import io, random
import pandas as pd
import time
from datetime import datetime
from sklearn.svm import SVR
import datetime
from skmultiflow.meta import AdaptiveRandomForestRegressor
from skmultiflow.trees import HoeffdingTreeRegressor
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import VotingRegressor

In [11]:
from skmultiflow.data import SEAGenerator
from skmultiflow.data import ConceptDriftStream
from skmultiflow.data import AGRAWALGenerator
from skmultiflow.data.regression_generator import RegressionGenerator

In [12]:
# stream = AGRAWALGenerator(s_r=1)      # create a stream
# stream.prepare_for_use()     # prepare the stream for use

In [13]:
stream = RegressionGenerator(n_samples=100000, n_features=6, n_targets=1, n_informative=4)

In [14]:
X,Y = stream.next_sample()


In [15]:
print(X)

[[-0.28186995  0.03209424  0.40912413  1.1234221   0.68003026  0.79396744]]


In [16]:
print(Y)

[146.51250452]


In [17]:
def pred_svr(x, y, x_test):
    svr_rbf = LinearRegression()
    y_rbf = svr_rbf.fit(np.array(x), np.ravel(y,order='C'))
    return svr_rbf.predict(x_test)

def pred_rff(x, y, x_test):
    arf_reg = AdaptiveRandomForestRegressor(random_state=123456)
    arf_reg.fit(np.array(x), np.ravel(y,order='C'))
    return arf_reg.predict(x_test)
 
def pred_hoe(x, y, x_test):
    #arf_reg = ExtraTreesRegressor(n_estimators=100, random_state=0)
    #arf_reg = linear_model.BayesianRidge()
    arf_reg = DecisionTreeRegressor(random_state=0)
    arf_reg.fit(np.array(x), np.ravel(y,order='C'))
    return arf_reg.predict(x_test)

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

def algorithms_pred(x_total,y_total):
    #x_total = df[["AMBIENT_TEMPERATURE","MODULE_TEMPERATURE","IRRADIATION"]].to_numpy().tolist()
    #y_total = df[["dA"]].to_numpy().tolist()
    x = x_total[:len(x_total)-1]
    y = y_total[:len(y_total)-1]
    
    x_test = x_total[len(x_total)-1:]
    y_test = y_total[len(y_total)-1:]
    print(len(x))
    print(len(y))
    y_pred_svr = pred_svr(x,y,x_test)
    y_pred_rff = pred_rff(x,y,x_test)
    y_pred_hoe = pred_hoe(x,y,x_test)
    
    ereg = VotingRegressor(estimators=[('gb', LinearRegression()), ('rf', AdaptiveRandomForestRegressor(random_state=123456)),
                                       ('lr', DecisionTreeRegressor(random_state=0))])
    ereg = ereg.fit(np.array(x), np.ravel(y,order='C'))
    y_pred_ensemb = ereg.predict(x_test)
    #y_pred_ensemb = (1/3)*np.add(np.add(y_pred_rff,y_pred_svr),y_pred_hoe)
#    if(y_test[0][0]!=0 and y_pred_svr!=0 and y_pred_rff!=0 and y_pred_hoe!=0):
#         if(y_test[0][0]==0):
#             y_test[0][0]+=1
#             y_pred_hoe[0]+=1
#             y_pred_rff[0]+=1
#             y_pred_svr[0]+=1
            
    print(y_test ,y_pred_svr, y_pred_rff, y_pred_hoe, y_pred_ensemb)
        #print(dt_start)
    MAPE_list.append([y_test[0],y_pred_svr[0]])
    MAPE_list2.append([y_test[0],y_pred_rff[0]])
    MAPE_list3.append([y_test[0],y_pred_hoe[0]])
    MAPE_list_ensemb.append([y_test[0],y_pred_ensemb[0]])
        

In [18]:
MAPE_list = list()
MAPE_list2 = list()
MAPE_list3 = list()
MAPE_list_ensemb = list()
for i in range(1,1000):
    x = list()
    y = list()
    for i in range(30):
        X,Y = stream.next_sample()
        x.append(X)
        y.append(Y)
    x = [arr[0].tolist() for arr in x]
    y = [arr[0].tolist() for arr in y]
    algorithms_pred(x,y)
MAPE_npArray = np.array(MAPE_list)
MAPE_npArray2 = np.array(MAPE_list2)
MAPE_npArray3 = np.array(MAPE_list3)
MAPE_npArray_ensemb = np.array(MAPE_list_ensemb)
#print(MAPE_npArray[:,0])
mape = MAPE(MAPE_npArray[:,0],MAPE_npArray[:,1])
mape2 = MAPE(MAPE_npArray2[:,0],MAPE_npArray2[:,1])
mape3 = MAPE(MAPE_npArray3[:,0],MAPE_npArray3[:,1])
mape_ensemb = MAPE(MAPE_npArray_ensemb[:,0],MAPE_npArray_ensemb[:,1])
print("MAPE for linear regression is: "+ str(mape))
print("MAPE for rff is: "+ str(mape2))
print("MAPE for decision tree is: "+ str(mape3))
print("MAPE for ensemble is: "+ str(mape_ensemb))
list1 = list()
list1.append(mape)
list1.append(mape2)
list1.append(mape3)
list1.append(mape_ensemb)



    


    

29
29
[-131.40449391387347] [-131.40449391] [-38.07212427] [-43.7355116] [-71.07070993]
29
29
[-66.09128903746884] [-66.09128904] [-7.65019485] [-71.2800615] [-48.34051513]
29
29
[17.207610165503084] [17.20761017] [28.33267075] [-15.39342948] [10.04895048]
29
29
[-121.50493997333591] [-121.50493997] [-7.81135951] [-72.58982717] [-67.30204222]
29
29
[-109.03913300502819] [-109.03913301] [-80.88535914] [-5.28174589] [-65.06874601]
29
29
[3.447624743847731] [3.44762474] [27.17612654] [-34.81706752] [-1.39777208]
29
29
[-162.7138213088822] [-162.71382131] [-121.87009862] [-101.64104032] [-128.74165341]
29
29
[111.3150163848677] [111.31501638] [20.37464024] [178.5773689] [103.42234184]
29
29
[-64.32173667111971] [-64.32173667] [-77.9469463] [-5.62375094] [-49.29747797]
29
29
[96.29919728107035] [96.29919728] [31.95885354] [35.95027964] [54.73611015]
29
29
[-197.69759852081] [-197.69759852] [-154.10083062] [-253.62186984] [-201.80676633]
29
29
[197.82660225364774] [197.82660225] [128.5160474

[-135.8359540472651] [-135.83595405] [-110.96717357] [-120.67091327] [-122.49134696]
29
29
[-45.91860220623247] [-45.91860221] [-49.43245485] [-132.26781374] [-75.87295693]
29
29
[51.35905958466883] [51.35905958] [22.32893166] [123.92863072] [65.87220732]
29
29
[-19.614481407308514] [-19.61448141] [43.19534942] [4.5544593] [9.37844244]
29
29
[74.01839945808072] [74.01839946] [34.31396803] [162.5334642] [90.28861056]
29
29
[65.76831392498796] [65.76831392] [8.81433537] [-43.16094147] [10.47390261]
29
29
[-200.3674052507935] [-200.36740525] [-84.31630119] [-15.20120275] [-99.9616364]
29
29
[-151.47206109658404] [-151.4720611] [-56.33639599] [-45.76105751] [-84.52317153]
29
29
[234.91030949669437] [234.9103095] [46.97147933] [203.94415462] [161.94198115]
29
29
[-148.797247690397] [-148.79724769] [-41.49155563] [-100.48748305] [-96.92542879]
29
29
[161.0880568134384] [161.08805681] [55.09037864] [80.30427784] [98.8275711]
29
29
[114.91774350797422] [114.91774351] [22.25852773] [126.1420474

[134.39747763284322] [134.39747763] [74.20068167] [68.14599575] [92.24805168]
29
29
[-120.19409179303494] [-120.19409179] [-89.37392944] [-30.19231435] [-79.92011186]
29
29
[-65.53921943561433] [-65.53921944] [-52.6964569] [-101.90908545] [-73.38158726]
29
29
[-3.1270567348216467] [-3.12705673] [29.70199467] [-2.04028234] [8.17821853]
29
29
[5.125986895943573] [5.1259869] [29.54608648] [-35.12759893] [-0.15184185]
29
29
[233.35737875509338] [233.35737876] [81.95202674] [-9.21506191] [102.03144786]
29
29
[-69.72780442077877] [-69.72780442] [-66.60424486] [-118.00185181] [-84.77796703]
29
29
[49.66074191372775] [49.66074191] [-16.43889361] [74.09057174] [35.77080668]
29
29
[-94.71005010439526] [-94.7100501] [-45.76912917] [-88.49603293] [-76.32507073]
29
29
[-91.72783598899548] [-91.72783599] [-46.27633147] [-47.56693973] [-61.85703573]
29
29
[8.000685522537841] [8.00068552] [-43.93880067] [34.54245287] [-0.46522076]
29
29
[-227.46825739275087] [-227.46825739] [-117.66041053] [-161.86153

[-205.01433497341583] [-205.01433497] [-92.87852689] [-92.31935545] [-130.07073911]
29
29
[84.7861116568586] [84.78611166] [52.73195583] [-12.4302934] [41.6959247]
29
29
[-132.65438027427751] [-132.65438027] [-63.61614167] [-99.06899501] [-98.44650565]
29
29
[29.09821277559255] [29.09821278] [-16.86820436] [-31.96990104] [-6.57996421]
29
29
[30.4868504469642] [30.48685045] [43.52832377] [-30.16047426] [14.61823332]
29
29
[-114.1350703114453] [-114.13507031] [-59.37715271] [-18.47125457] [-63.99449253]
29
29
[-181.4691023417846] [-181.46910234] [-149.72068598] [-283.18935823] [-204.79304885]
29
29
[62.246007965670536] [62.24600797] [57.94677571] [151.23959992] [90.4774612]
29
29
[140.3919554000213] [140.3919554] [58.55381091] [224.27855664] [141.07477432]
29
29
[47.84573088708421] [47.84573089] [19.48532605] [10.96157815] [26.09754503]
29
29
[56.00959329994361] [56.0095933] [51.34859009] [-7.20156329] [33.38554003]
29
29
[-140.98341198070244] [-140.98341198] [-51.25453231] [-213.2336163

[-95.70736644429904] [-95.70736644] [2.81705205] [-16.99415089] [-36.6281551]
29
29
[-21.03482365472667] [-21.03482365] [31.08872647] [-14.76860135] [-1.57156618]
29
29
[-24.664874359921978] [-24.66487436] [-8.89937956] [-33.06993199] [-22.2113953]
29
29
[-43.846971911852265] [-43.84697191] [26.20429238] [-106.68426812] [-41.44231588]
29
29
[-137.65477810926853] [-137.65477811] [-52.31045234] [-101.30700986] [-97.09074677]
29
29
[119.00863977566581] [119.00863978] [93.0802483] [30.57275974] [80.88721594]
29
29
[-141.26885549180702] [-141.26885549] [12.32791898] [-64.75004002] [-64.56365885]
29
29
[-59.75993802851158] [-59.75993803] [-45.28118253] [-78.44066656] [-61.16059571]
29
29
[9.027240129877885] [9.02724013] [-10.50170095] [-46.67005257] [-16.04817113]
29
29
[40.52396542903388] [40.52396543] [3.31657456] [-245.18006475] [-67.11317492]
29
29
[136.28435726711191] [136.28435727] [87.39332591] [137.60382433] [120.42716917]
29
29
[-41.84464749076639] [-41.84464749] [-95.18738192] [-8.

...

[-84.0357719162621] [-84.03577192] [8.29973335] [-72.76242962] [-49.4994894]
29
29
[3.4248388747282323] [3.42483887] [35.11936008] [-20.03298537] [6.17040453]
29
29
[-7.179038186295275] [-7.17903819] [-19.93778315] [-46.05970413] [-24.39217515]
29
29
[65.85826318255481] [65.85826318] [44.85471843] [48.66271229] [53.1252313]
29
29
[-15.797948310890686] [-15.79794831] [-8.75271758] [20.07641189] [-1.491418]
29
29
[8.504332564021666] [8.50433256] [-5.03162733] [95.12496642] [32.86589055]
29
29
[-79.85042458556583] [-79.85042459] [-51.38539223] [8.97820869] [-40.75253604]
29
29
[50.03017853216573] [50.03017853] [-1.46194693] [33.53046945] [27.36623368]
29
29
[-92.213098577087] [-92.21309858] [-36.2096965] [-191.89733819] [-106.77337776]
29
29
[80.94405544921209] [80.94405545] [51.75796999] [59.09394355] [63.93198966]
29
29
[257.41817251910294] [257.41817252] [97.2616255] [58.83112741] [137.83697514]
29
29
[65.83473005750697] [65.83473006] [83.10197813] [24.9070186] [57.94790893]
29
29
[60.

[22.995983443717634] [22.99598344] [-40.50546783] [105.17649409] [29.22233657]
29
29
[158.10185883832443] [158.10185884] [53.79513225] [9.72133254] [73.87277454]
29
29
[17.310917194588015] [17.31091719] [-3.88915055] [-210.92313562] [-65.83378966]
29
29
[-177.73926347150962] [-177.73926347] [-112.10062037] [-118.1867091] [-136.00886431]
29
29
[-183.33968981311398] [-183.33968981] [-106.06541003] [-110.38822929] [-133.26444305]
29
29
[376.1604720287571] [376.16047203] [230.97054544] [112.00463215] [239.71188321]
29
29
[118.73320952056798] [118.73320952] [80.8322036] [-8.22728246] [63.77937689]
29
29
[103.77688362503329] [103.77688363] [25.53223932] [91.16212766] [73.49041687]
29
29
[-18.617221784533616] [-18.61722178] [-33.57299111] [-61.26679573] [-37.81900287]
29
29
[170.30453197034146] [170.30453197] [107.63737036] [206.0808939] [161.34093208]
29
29
[11.097534457816288] [11.09753446] [-33.23259483] [75.19300895] [17.68598286]
29
29
[-83.05306896168506] [-83.05306896] [-38.08205058] [

[17.009950552454956] [17.00995055] [-1.38379213] [34.21365639] [16.61327161]
29
29
[-57.52705531882301] [-57.52705532] [-48.86240815] [-68.96709004] [-58.4521845]
29
29
[-70.62318760171402] [-70.6231876] [4.21520093] [-30.11997435] [-32.17598701]
29
29
[-292.71244901683434] [-292.71244902] [-44.09104118] [-13.31715566] [-116.70688195]
29
29
[100.0509565402131] [100.05095654] [2.7252278] [132.6290193] [78.46840121]
29
29
[108.12673910683516] [108.12673911] [46.73042732] [168.13917569] [107.66544737]
29
29
[-65.5000201656073] [-65.50002017] [-40.31306143] [-105.41500091] [-70.40936084]
29
29
[33.02357735507361] [33.02357736] [-74.24971433] [40.37837677] [-0.28258673]
29
29
[10.614230096809255] [10.6142301] [42.84112483] [-13.81638585] [13.21298969]
29
29
[-159.22509198489965] [-159.22509198] [-100.65945269] [-138.01986091] [-132.63480186]
29
29
[-44.93872593470145] [-44.93872593] [-19.6763535] [5.25685596] [-19.78607449]
29
29
[161.81870436951644] [161.81870437] [121.4331795] [-26.226427

[193.38258409798684] [193.3825841] [126.54120711] [109.55186856] [143.15855325]
29
29
[-29.2652336985524] [-29.2652337] [96.12994245] [2.53809955] [23.13426943]
29
29
[158.59693294046983] [158.59693294] [83.33041438] [89.22357464] [110.38364065]
29
29
[-67.61281645028936] [-67.61281645] [-38.36928133] [-24.8170088] [-43.59970219]
29
29
[63.69256898139664] [63.69256898] [47.34227449] [148.03197826] [86.35560724]
29
29
[85.3744600717248] [85.37446007] [25.44514433] [30.59203834] [47.13721425]
29
29
[-43.30833457400169] [-43.30833457] [-69.37187084] [43.44839122] [-23.0772714]
29
29
[64.31764222315306] [64.31764222] [24.57736566] [105.17154894] [64.68885227]
29
29
[-61.409311046351704] [-61.40931105] [-70.42959437] [-53.39094894] [-61.74328479]
29
29
[35.304004360583136] [35.30400436] [83.42675292] [43.4271984] [54.05265189]
29
29
[-250.59643696643337] [-250.59643697] [-118.19486994] [-206.50082688] [-191.7640446]
29
29
[-108.76939856714776] [-108.76939857] [-89.37492961] [-60.6327971] [-

[-94.15289911793833] [-94.15289912] [-30.14697162] [-15.47061837] [-46.59016304]
29
29
[-230.18347444673503] [-230.18347445] [-70.13119027] [-187.10723568] [-162.4739668]
29
29
[-223.13885976300773] [-223.13885976] [-149.78830155] [-227.07424028] [-200.0004672]
29
29
[-117.2131380632483] [-117.21313806] [-98.93819822] [84.16040012] [-43.99697872]
29
29
[155.7519950992727] [155.7519951] [69.90362789] [115.01346054] [113.55636118]
29
29
[14.07323333850222] [14.07323334] [30.00000612] [126.0025061] [56.69191518]
29
29
[54.0825824155165] [54.08258242] [6.11805215] [-36.47918147] [7.90715103]
29
29
[88.28241963215638] [88.28241963] [47.0841151] [53.50310329] [62.95654601]
29
29
[-58.369636285157526] [-58.36963629] [-79.55953598] [-7.98255265] [-48.63724164]
29
29
[-47.0955841141877] [-47.09558411] [-21.17162337] [-9.44824029] [-25.90514926]
29
29
[-132.0868711964614] [-132.0868712] [-65.33452144] [-144.64310785] [-114.02150016]
29
29
[61.028637803034805] [61.0286378] [2.44504042] [174.63539